In [ ]:
import pandas as pd
BASE_URL = "https://voyager.online/api/staking/validators"
page_size = 50

# Iterate over pages 1 to 3 (adjust the range as needed)
for p in range(1, 4):
    url = f"{BASE_URL}?search=&p={p}&ps={page_size}"
    data = requests.get(url).json()
    print(f"Page {p}:")
    print(data)


In [4]:
import requests
import psycopg2

conn = psycopg2.connect(
    dbname="mydatabase",
    user="user",
    password="password",
    host="localhost",
    port="5432"  # typically 5432
)
cur = conn.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS validators (
    address TEXT PRIMARY KEY,
    tokenAddress TEXT,
    isVerified BOOLEAN,
    totalStake NUMERIC,             -- changed from TEXT to NUMERIC
    totalStakePercentage NUMERIC,   -- changed from INTEGER to NUMERIC if decimals are needed
    ownStake NUMERIC,               -- changed from TEXT to NUMERIC
    delegatedStake NUMERIC,         -- changed from TEXT to NUMERIC
    totalDelegators INTEGER,
    revenueShare NUMERIC,           -- changed from INTEGER to NUMERIC if needed
    apr NUMERIC,                    -- changed from INTEGER to NUMERIC if decimals are needed
    rank TEXT,
    name TEXT,
    imgSrc TEXT,
    startTime BIGINT,
    poolAddress TEXT,
    stakerAddress TEXT
);
"""
cur.execute(create_table_query)
conn.commit()

BASE_URL = "https://voyager.online/api/staking/validators"
page_size = 100

for p in range(1, 4):
    url = f"{BASE_URL}?search=&p={p}&ps={page_size}"
    response = requests.get(url)
    data = response.json()
    items = data.get("items", [])
    
    for item in items:
        query = """
        INSERT INTO validators (
            address, tokenAddress, isVerified, totalStake, totalStakePercentage,
            ownStake, delegatedStake, totalDelegators, revenueShare, apr, rank,
            name, imgSrc, startTime, poolAddress, stakerAddress
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (address) DO NOTHING;
        """
        values = (
            item.get("address"),
            item.get("tokenAddress"),
            item.get("isVerified"),
            item.get("totalStake"),
            item.get("totalStakePercentage"),
            item.get("ownStake"),
            item.get("delegatedStake"),
            item.get("totalDelegators"),
            item.get("revenueShare"),
            item.get("apr"),
            item.get("rank"),
            item.get("name"),
            item.get("imgSrc"),
            item.get("startTime"),
            item.get("poolAddress"),
            item.get("stakerAddress")
        )
        cur.execute(query, values)
    
    conn.commit()
    print(f"Inserted data from page {p}")

cur.close()
conn.close()

Inserted data from page 1
Inserted data from page 2
Inserted data from page 3


In [3]:
import requests
import psycopg2


conn = psycopg2.connect(
    dbname="mydatabase",
    user="user",
    password="password",
    host="localhost",
    port="5432" 
)
cur = conn.cursor()

query_get_avg_delegated_stake_top_10 = """SELECT AVG(delegatedStake) 
FROM (
    SELECT delegatedStake 
    FROM validators 
    ORDER BY delegatedStake DESC
    limit 10
) subquery;"""


cur.execute(query_get_avg_delegated_stake_top_10)
result = cur.fetchall()
print(result[0][0] / 10**18)

cur.close()
conn.close()


15209196.437081446278959945


In [12]:
import requests

selector = "0x1e888a1026b19c8c0b57c72d63ed1737106aa10034105b980ba117bd0c29fe1"

RPC_URL = "https://free-rpc.nethermind.io/mainnet-juno/v0_7"

contract_address = "0x04718f5a0fc34cc1af16a1cdee98ffb20c31f5cd61d6ab07201858f4287c938d"
owner = "0x030016e7beABe590593a1958e31D2aFa2D06754Ea8468e3e720c781c3cA690d6"
spender = "0x071c4334c5b387cbc84a3a62b16c6eb0481bf596054d18fd983e5f3fc27e489b"

payload = {
    "jsonrpc": "2.0",
    "method": "starknet_call",
    "params": [
        {
            "contract_address": contract_address,
            "entry_point_selector": selector,
            "calldata": [owner, spender]
        },
        "latest"
    ],
    "id": 1
}

r = requests.post(RPC_URL, json=payload)
result = r.json()
print(result)

if "result" in result:
    allowance_hex = result["result"][0]
    print("Allowance (hex):", allowance_hex)
    print("Allowance (dec):", int(allowance_hex, 16))
else:
    print("Error:", result.get("error"))


{'jsonrpc': '2.0', 'result': ['0xde0b6b3a7640000', '0x0'], 'id': 1}
Allowance (hex): 0xde0b6b3a7640000
Allowance (dec): 1000000000000000000
